Earned Value Cost
The portion of the project baseline total cost of an activity or all activities in the project that are actually completed as of the project data date.

Budget at completion is calculated from the project baseline.

Calculated as Budget At Completion multiplied by Performance Percent Complete. The method for computing performance percent complete depends on the Earned Value technique selected for the activity's WBS.

Budget At Completion
The planned total cost through activity or project completion.

Calculated as Planned Labor Cost plus Planned Nonlabor Cost plus Planned Expense Cost plus Planned Material Cost.

Planned Labor Cost (Activities)
The planned costs for all labor resources assigned to the activity.

If no resources are assigned, calculated as Activity Planned Labor Units multiplied by Project Default Price divided by Time.

Planned Material Cost
The planned cost for all material resources assigned to the activity, project, or EPS.

Planned Nonlabor Cost
The planned costs for all nonlabor resources assigned to the activity.

If no resources are assigned, calculated as Activity Planned Nonlabor Units multiplied by Project Default Price divided by Time.

Planned Expense Cost
The planned total cost of all expenses associated with the activity, project, or EPS.

In [31]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [32]:
# specify the file path, you might need to adjust this
file_path = "Task_Calculation.xlsx"
# Load the Task data into a DataFrame
task_calculation_df = pd.read_excel(file_path, sheet_name='Task_Calculation')

In [33]:
# specify the file path, you might need to adjust this
file_path = "TST00-Update06.xlsx"
# Load the Task data into a DataFrame
expenses_df = pd.read_excel(file_path, sheet_name='PROJCOST')

In [34]:
# specify the file path, you might need to adjust this
file_path = "TST00-Update06.xlsx"
# Load the Task data into a DataFrame
resources_cost_df = pd.read_excel(file_path, sheet_name='TASKRSRC')

In [35]:
# Assuming you have 'task_calculation_df' and 'expenses_df' dataframes. Make sure that 'task_id' is a common column in both dataframes.
# Below is the code for performing left join:
merged_df = pd.merge(task_calculation_df, expenses_df, on='task_id', how='left')
# Next, calculate target_cost for each task_id. Use groupby() for grouping task_id, and the sum() function to calculate the total target_cost:
grouped_df = merged_df.groupby('task_id')['target_cost'].sum().reset_index()
# Finally, let's put the result into temp_cost_df tensorflow:
temp_cost_df = grouped_df.rename(columns={'target_cost': 'expenses_cost'})

In [36]:
temp_cost_df

,task_id,expenses_cost
0,316508,1000.0
1,316509,1000.0
2,316510,250.0


In [37]:
# First, let's perform the left join:
merged_df = pd.merge(task_calculation_df, resources_cost_df, on='task_id', how='left')
# Next, let's group by task_id and sum target_cost:
grouped_df = merged_df.groupby('task_id')['target_cost'].sum().reset_index()
# Rename the 'target_cost' column to 'resources_cost':
grouped_df = grouped_df.rename(columns={'target_cost': 'resources_cost'})
# Now, let's add this new column to our 'temp_cost_df':
temp_cost_df = pd.merge(temp_cost_df, grouped_df, on='task_id', how='left')

In [38]:
temp_cost_df

,task_id,expenses_cost,resources_cost
0,316508,1000.0,0.0
1,316509,1000.0,0.0
2,316510,250.0,750.0


In [39]:
import pandas as pd
import numpy as np

# left join on the task_id
merged_df = pd.merge(task_calculation_df, resources_cost_df, on='task_id', how='left')

# groupby task_id and rsrc_type and sum target_cost
grouped_df = merged_df.groupby(['task_id', 'rsrc_type'])['target_cost'].sum().reset_index()

# create a pivot table to map rsrc_type to cost columns
pivot_df = grouped_df.pivot(index='task_id', columns='rsrc_type', values='target_cost').reset_index()
pivot_df.columns.name = None
pivot_df = pivot_df.rename(columns={'RT_Labor': 'labor_cost', 'RT_Equip': 'nonlabor_cost', 'RT_Mat': 'material_cost'})

pivot_df = pivot_df.replace(np.nan, 0)

# left join pivot_df with original temp_cost_df
temp_cost_df = pd.merge(temp_cost_df, pivot_df, on='task_id', how='left')

In [40]:
temp_cost_df

,task_id,expenses_cost,resources_cost,nonlabor_cost,labor_cost,material_cost
0,316508,1000.0,0.0,NaN,NaN,NaN
1,316509,1000.0,0.0,NaN,NaN,NaN
2,316510,250.0,750.0,250.0,250.0,250.0


In [41]:
import pandas as pd

# Given that 'task_calculation_df' and 'temp_cost_df' are your current DataFrames.

# Adding columns to 'task_calculation_df' from 'temp_cost_df'.
task_calculation_df = pd.merge(task_calculation_df, temp_cost_df[
    ['task_id', 'labor_cost', 'nonlabor_cost', 'material_cost', 'expenses_cost']], on='task_id', how='left')

# Replacing NaN values with 0.
task_calculation_df[['labor_cost', 'nonlabor_cost', 'material_cost', 'expenses_cost']] = task_calculation_df[
    ['labor_cost', 'nonlabor_cost', 'material_cost', 'expenses_cost']].fillna(0)

# Creating a new column "Budget_Total_Cost" which is the sum of the values in the added columns ('labor_cost', 'nonlabor_cost', 'material_cost', 'expenses_cost').
task_calculation_df['Budget_Total_Cost'] = task_calculation_df[
    ['labor_cost', 'nonlabor_cost', 'material_cost', 'expenses_cost']].sum(axis=1)

In [42]:
import pandas as pd

# Continue from the previous operations

# Rename the columns
task_calculation_df = task_calculation_df.rename(columns={
    'labor_cost': 'Budgeted_Labor_Cost',
    'nonlabor_cost': 'Budgeted_Nonlabor_Cost',
    'material_cost': 'Budgeted_Material_Cost',
    'expenses_cost': 'Budgeted_Expenses_Cost'
})

In [43]:
task_calculation_df

,task_id,proj_id,wbs_id,Activity_%_Complete,Performance_%_Complete,Budgeted_Labor_Cost,Budgeted_Nonlabor_Cost,Budgeted_Material_Cost,Budgeted_Expenses_Cost,Budget_Total_Cost
0,316508,1481,85132,27,27,0.0,0.0,0.0,1000.0,1000.0
1,316509,1481,85132,23,23,0.0,0.0,0.0,1000.0,1000.0
2,316510,1481,85132,25,25,250.0,250.0,250.0,250.0,1000.0


In [44]:
import pandas as pd

# First, let's left join task_calculation_df and expenses_df using the merge function in pandas
merged_df = task_calculation_df.merge(expenses_df, on='task_id', how='left')

# Next, group by task_id and sum up the remain_cost. The result is a DataFrame with task_id and the corresponding sum of remain_cost
grouped_df = merged_df.groupby('task_id')['remain_cost'].sum().reset_index()

# Rename the second column to Remaining_Expenses_Cost
grouped_df = grouped_df.rename(columns={'remain_cost': 'Remaining_Expenses_Cost'})

# Now, let's create temp_cost_df contains the task_id and Remaining_Expenses_Cost
temp_cost_df = pd.DataFrame(grouped_df)

In [45]:
temp_cost_df

,task_id,Remaining_Expenses_Cost
0,316508,800.0
1,316509,780.0
2,316510,175.0


In [46]:
import pandas as pd

# Left join task_calculation_df and resources_cost_df on task_id
merged_df = task_calculation_df.merge(resources_cost_df, on='task_id', how='left')

# Now, let's handle each rsrc_type individually

# For 'RT_Labor'
labor_df = merged_df[merged_df['rsrc_type'] == 'RT_Labor']
grouped_labor_df = labor_df.groupby(['task_id'])['remain_cost'].sum().reset_index()
grouped_labor_df = grouped_labor_df.rename(columns={'remain_cost': 'Remaining_Labor_Cost'})

# For 'RT_Equip'
equip_df = merged_df[merged_df['rsrc_type'] == 'RT_Equip']
grouped_equip_df = equip_df.groupby(['task_id'])['remain_cost'].sum().reset_index()
grouped_equip_df = grouped_equip_df.rename(columns={'remain_cost': 'Remaining_Nonlabor_Cost'})

# For 'RT_Mat'
mat_df = merged_df[merged_df['rsrc_type'] == 'RT_Mat']
grouped_mat_df = mat_df.groupby(['task_id'])['remain_cost'].sum().reset_index()
grouped_mat_df = grouped_mat_df.rename(columns={'remain_cost': 'Remaining_Material_Cost'})

# Now, let's add these new columns to the existing temp_cost_df dataframe
temp_cost_df = temp_cost_df.merge(grouped_labor_df, on='task_id', how='left')
temp_cost_df = temp_cost_df.merge(grouped_equip_df, on='task_id', how='left')
temp_cost_df = temp_cost_df.merge(grouped_mat_df, on='task_id', how='left')

# replace any NaN values with 0
temp_cost_df[['Remaining_Labor_Cost', 'Remaining_Nonlabor_Cost', 'Remaining_Material_Cost']] = temp_cost_df[
    ['Remaining_Labor_Cost', 'Remaining_Nonlabor_Cost', 'Remaining_Material_Cost']].fillna(0)

In [47]:
temp_cost_df

,task_id,Remaining_Expenses_Cost,Remaining_Labor_Cost,Remaining_Nonlabor_Cost,Remaining_Material_Cost
0,316508,800.0,0.0,0.0,0.0
1,316509,780.0,0.0,0.0,0.0
2,316510,175.0,150.0,175.0,250.0


In [48]:
import pandas as pd

# Let's add the columns 'Remaining_Labor_Cost', 'Remaining_Nonlabor_Cost', 'Remaining_Material_Cost', and 'Remaining_Expenses_Cost' from temp_cost_df to task_calculation_df
task_calculation_df = pd.merge(task_calculation_df, temp_cost_df[
    ['task_id', 'Remaining_Labor_Cost', 'Remaining_Nonlabor_Cost', 'Remaining_Material_Cost',
     'Remaining_Expenses_Cost']], on='task_id', how='left')

# Convert any NaN values in these columns to 0
task_calculation_df[
    ['Remaining_Labor_Cost', 'Remaining_Nonlabor_Cost', 'Remaining_Material_Cost', 'Remaining_Expenses_Cost']] = \
    task_calculation_df[
        ['Remaining_Labor_Cost', 'Remaining_Nonlabor_Cost', 'Remaining_Material_Cost', 'Remaining_Expenses_Cost']].fillna(0)

# Lastly, let's add the new 'Remaining_Total_Cost' column, which is the sum of the 'Remaining_Labor_Cost', 'Remaining_Nonlabor_Cost', 'Remaining_Material_Cost', and 'Remaining_Expenses_Cost' columns
task_calculation_df['Remaining_Total_Cost'] = task_calculation_df[
    ['Remaining_Labor_Cost', 'Remaining_Nonlabor_Cost', 'Remaining_Material_Cost', 'Remaining_Expenses_Cost']].sum(
    axis=1)

In [49]:
task_calculation_df

,task_id,proj_id,wbs_id,Activity_%_Complete,Performance_%_Complete,Budgeted_Labor_Cost,Budgeted_Nonlabor_Cost,Budgeted_Material_Cost,Budgeted_Expenses_Cost,Budget_Total_Cost,Remaining_Labor_Cost,Remaining_Nonlabor_Cost,Remaining_Material_Cost,Remaining_Expenses_Cost,Remaining_Total_Cost
0,316508,1481,85132,27,27,0.0,0.0,0.0,1000.0,1000.0,0.0,0.0,0.0,800.0,800.0
1,316509,1481,85132,23,23,0.0,0.0,0.0,1000.0,1000.0,0.0,0.0,0.0,780.0,780.0
2,316510,1481,85132,25,25,250.0,250.0,250.0,250.0,1000.0,150.0,175.0,250.0,175.0,750.0


In [50]:
import pandas as pd

# First, let's left join task_calculation_df and expenses_df using the merge function in pandas
merged_df = task_calculation_df.merge(expenses_df, on='task_id', how='left')

# Next, group by task_id and sum up the act_cost. The result is a DataFrame with task_id and the corresponding sum of act_cost
grouped_df = merged_df.groupby('task_id')['act_cost'].sum().reset_index()

# Rename the second column to Actual_Expenses_Cost
grouped_df = grouped_df.rename(columns={'act_cost': 'Actual_Expenses_Cost'})

# Now, let's create temp_cost_df contains the task_id and Actual_Expenses_Cost
temp_cost_df = pd.DataFrame(grouped_df)

In [51]:
temp_cost_df

,task_id,Actual_Expenses_Cost
0,316508,200.0
1,316509,520.0
2,316510,75.0


In [52]:
# Add a new column 'act_cost' to the dataframe
resources_cost_df['act_cost'] = resources_cost_df['act_reg_cost'] + resources_cost_df['act_ot_cost']

In [53]:
resources_cost_df

,%F,taskrsrc_id,task_id,proj_id,cost_qty_link_flag,role_id,acct_id,rsrc_id,pobs_id,skill_level,remain_qty,target_qty,remain_qty_per_hr,target_lag_drtn_hr_cnt,target_qty_per_hr,act_ot_qty,act_reg_qty,relag_drtn_hr_cnt,ot_factor,cost_per_qty,target_cost,act_reg_cost,act_ot_cost,remain_cost,act_start_date,act_end_date,restart_date,reend_date,target_start_date,target_end_date,rem_late_start_date,rem_late_end_date,rollup_dates_flag,target_crv,remain_crv,actual_crv,ts_pend_act_end_flag,guid,rate_type,act_this_per_cost,act_this_per_qty,curv_id,rsrc_type,cost_per_qty_source_type,create_user,create_date,has_rsrchours,taskrsrc_sum_id,act_cost
0,%R,443523,316510,1481,N,NaN,NaN,1671,NaN,NaN,60,100,0.75,0,1,0,20,0,NaN,0.0,250.0,150.0,0.0,150.0,2024-06-01 07:00,NaN,2024-06-06 13:00,2024-06-20 13:00,2024-06-01 07:00,2024-06-11 17:00,2024-06-06 13:00,2024-06-20 13:00,Y,NaN,NaN,NaN,N,6URNtrGH6UG7SVPEiOFQ5A,COST_PER_QTY,150.0,20,NaN,RT_Labor,ST_Rsrc,NotPrmUser,2024-10-25 22:36,NaN,NaN,150.0
1,%R,443524,316510,1481,N,NaN,NaN,1672,NaN,NaN,60,100,0.75,0,1,0,20,0,NaN,0.0,250.0,75.0,0.0,175.0,2024-06-01 07:00,NaN,2024-06-06 13:00,2024-06-20 13:00,2024-06-01 07:00,2024-06-11 17:00,2024-06-06 13:00,2024-06-20 13:00,Y,NaN,NaN,NaN,N,9nY1zKCM9U+RZxYZlllmsw,COST_PER_QTY,75.0,20,NaN,RT_Equip,ST_Rsrc,NotPrmUser,2024-10-25 22:36,NaN,NaN,75.0
2,%R,443525,316510,1481,N,NaN,NaN,1673,NaN,NaN,80,100,1.00,0,1,0,20,0,0.0,0.0,250.0,200.0,0.0,250.0,2024-06-01 07:00,NaN,2024-06-06 13:00,2024-06-20 13:00,2024-06-01 07:00,2024-06-11 17:00,2024-06-06 13:00,2024-06-20 13:00,Y,NaN,NaN,NaN,N,9JfuIDOpIUaOORAqa6IVrg,COST_PER_QTY,200.0,20,NaN,RT_Mat,ST_Rsrc,NotPrmUser,2024-10-25 22:36,NaN,NaN,200.0


In [54]:
import pandas as pd

# Left join task_calculation_df and resources_cost_df on task_id
merged_df = task_calculation_df.merge(resources_cost_df, on='task_id', how='left')

# Handle 'RT_Labor' rsrc_type
labor_df = merged_df[merged_df['rsrc_type'] == 'RT_Labor']
grouped_labor_df = labor_df.groupby('task_id')['act_cost'].sum().reset_index()
grouped_labor_df = grouped_labor_df.rename(columns={'act_cost': 'Actual_Labor_Cost'})

# Handle 'RT_Equip' rsrc_type
equip_df = merged_df[merged_df['rsrc_type'] == 'RT_Equip']
grouped_equip_df = equip_df.groupby('task_id')['act_cost'].sum().reset_index()
grouped_equip_df = grouped_equip_df.rename(columns={'act_cost': 'Actual_Nonlabor_Cost'})

# Handle 'RT_Mat' rsrc_type
mat_df = merged_df[merged_df['rsrc_type'] == 'RT_Mat']
grouped_mat_df = mat_df.groupby('task_id')['act_cost'].sum().reset_index()
grouped_mat_df = grouped_mat_df.rename(columns={'act_cost': 'Actual_Material_Cost'})

# Merge to temp_cost_df
temp_cost_df = temp_cost_df.merge(grouped_labor_df, on='task_id', how='left')
temp_cost_df = temp_cost_df.merge(grouped_equip_df, on='task_id', how='left')
temp_cost_df = temp_cost_df.merge(grouped_mat_df, on='task_id', how='left')

# Replace NaN values with 0
temp_cost_df[['Actual_Labor_Cost', 'Actual_Nonlabor_Cost', 'Actual_Material_Cost']] = temp_cost_df[
    ['Actual_Labor_Cost', 'Actual_Nonlabor_Cost', 'Actual_Material_Cost']].fillna(0)

In [55]:
temp_cost_df

,task_id,Actual_Expenses_Cost,Actual_Labor_Cost,Actual_Nonlabor_Cost,Actual_Material_Cost
0,316508,200.0,0.0,0.0,0.0
1,316509,520.0,0.0,0.0,0.0
2,316510,75.0,150.0,75.0,200.0


In [56]:
import pandas as pd

# Add the columns 'Actual_Labor_Cost', 'Actual_Nonlabor_Cost', 'Actual_Material_Cost', and 'Actual_Expenses_Cost' from temp_cost_df to task_calculation_df
task_calculation_df = pd.merge(task_calculation_df, temp_cost_df[
    ['task_id', 'Actual_Labor_Cost', 'Actual_Nonlabor_Cost', 'Actual_Material_Cost', 'Actual_Expenses_Cost']],
                               on='task_id', how='left')

# Convert any NaN values to 0
task_calculation_df[['Actual_Labor_Cost', 'Actual_Nonlabor_Cost', 'Actual_Material_Cost', 'Actual_Expenses_Cost']] = \
    task_calculation_df[
        ['Actual_Labor_Cost', 'Actual_Nonlabor_Cost', 'Actual_Material_Cost', 'Actual_Expenses_Cost']].fillna(0)

# Add the 'Actual_Total_Cost' column, which is the sum of the 'Actual_Labor_Cost', 'Actual_Nonlabor_Cost', 'Actual_Material_Cost', and 'Actual_Expenses_Cost' columns
task_calculation_df['Actual_Total_Cost'] = task_calculation_df[
    ['Actual_Labor_Cost', 'Actual_Nonlabor_Cost', 'Actual_Material_Cost', 'Actual_Expenses_Cost']].sum(axis=1)

In [57]:
task_calculation_df

,task_id,proj_id,wbs_id,Activity_%_Complete,Performance_%_Complete,Budgeted_Labor_Cost,Budgeted_Nonlabor_Cost,Budgeted_Material_Cost,Budgeted_Expenses_Cost,Budget_Total_Cost,Remaining_Labor_Cost,Remaining_Nonlabor_Cost,Remaining_Material_Cost,Remaining_Expenses_Cost,Remaining_Total_Cost,Actual_Labor_Cost,Actual_Nonlabor_Cost,Actual_Material_Cost,Actual_Expenses_Cost,Actual_Total_Cost
0,316508,1481,85132,27,27,0.0,0.0,0.0,1000.0,1000.0,0.0,0.0,0.0,800.0,800.0,0.0,0.0,0.0,200.0,200.0
1,316509,1481,85132,23,23,0.0,0.0,0.0,1000.0,1000.0,0.0,0.0,0.0,780.0,780.0,0.0,0.0,0.0,520.0,520.0
2,316510,1481,85132,25,25,250.0,250.0,250.0,250.0,1000.0,150.0,175.0,250.0,175.0,750.0,150.0,75.0,200.0,75.0,500.0


In [58]:
# Add the new columns to task_calculation_df
task_calculation_df['At_Completion_Labor_Cost'] = task_calculation_df['Actual_Labor_Cost'] + task_calculation_df[
    'Remaining_Labor_Cost']
task_calculation_df['At_Completion_Nonlabor_Cost'] = task_calculation_df['Actual_Nonlabor_Cost'] + task_calculation_df[
    'Remaining_Nonlabor_Cost']
task_calculation_df['At_Completion_Material_Cost'] = task_calculation_df['Actual_Material_Cost'] + task_calculation_df[
    'Remaining_Material_Cost']
task_calculation_df['At_Completion_Expenses_Cost'] = task_calculation_df['Actual_Expenses_Cost'] + task_calculation_df[
    'Remaining_Expenses_Cost']

# Now, let's add the At_Completion_Total_Cost column, which is the sum of the At_Completion_Labor_Cost, At_Completion_Nonlabor_Cost, At_Completion_Material_Cost, and At_Completion_Expenses_Cost columns
task_calculation_df['At_Completion_Total_Cost'] = task_calculation_df[
    ['At_Completion_Labor_Cost', 'At_Completion_Nonlabor_Cost', 'At_Completion_Material_Cost',
     'At_Completion_Expenses_Cost']].sum(axis=1)

In [59]:
task_calculation_df

,task_id,proj_id,wbs_id,Activity_%_Complete,Performance_%_Complete,Budgeted_Labor_Cost,Budgeted_Nonlabor_Cost,Budgeted_Material_Cost,Budgeted_Expenses_Cost,Budget_Total_Cost,Remaining_Labor_Cost,Remaining_Nonlabor_Cost,Remaining_Material_Cost,Remaining_Expenses_Cost,Remaining_Total_Cost,Actual_Labor_Cost,Actual_Nonlabor_Cost,Actual_Material_Cost,Actual_Expenses_Cost,Actual_Total_Cost,At_Completion_Labor_Cost,At_Completion_Nonlabor_Cost,At_Completion_Material_Cost,At_Completion_Expenses_Cost,At_Completion_Total_Cost
0,316508,1481,85132,27,27,0.0,0.0,0.0,1000.0,1000.0,0.0,0.0,0.0,800.0,800.0,0.0,0.0,0.0,200.0,200.0,0.0,0.0,0.0,1000.0,1000.0
1,316509,1481,85132,23,23,0.0,0.0,0.0,1000.0,1000.0,0.0,0.0,0.0,780.0,780.0,0.0,0.0,0.0,520.0,520.0,0.0,0.0,0.0,1300.0,1300.0
2,316510,1481,85132,25,25,250.0,250.0,250.0,250.0,1000.0,150.0,175.0,250.0,175.0,750.0,150.0,75.0,200.0,75.0,500.0,300.0,250.0,450.0,250.0,1250.0


In [60]:
# Assuming Task_Calculation_df is your DataFrame
task_calculation_path = 'Task_Calculation.xlsx'  # you may want to include full path here
with pd.ExcelWriter(task_calculation_path, engine='openpyxl') as writer:
    task_calculation_df.to_excel(writer, sheet_name='Task_Calculation', index=False)